In [68]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [70]:
%%sql
drop table if exists Sales2;
CREATE TABLE Sales2
(year INTEGER NOT NULL , 
 sale INTEGER NOT NULL , 
 PRIMARY KEY (year));

INSERT INTO Sales2 VALUES (1990, 50);
INSERT INTO Sales2 VALUES (1992, 50);
INSERT INTO Sales2 VALUES (1993, 52);
INSERT INTO Sales2 VALUES (1994, 55);
INSERT INTO Sales2 VALUES (1997, 55);

select *
from Sales2

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
5 rows affected.


year,sale
1990,50
1992,50
1993,52
1994,55
1997,55


## やりたいこと
+ 前年と比較できるレコードを抽出する(1993年は1992年のレコードがあるのでOK)

In [71]:
%%sql
select year, sale
from Sales2 as s1
where exists (
    select *
    from Sales2 as s2
    where s1.year - 1 = s2.year
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


year,sale
1993,52
1994,55


In [78]:
%%sql
-- window関数のrowsを意識(明示的に前の行と比較する)して、相関サブクエリ
select year, sale
from Sales2 as s1
where exists (
    select *
    from sales2 as s2
    -- s1より前のデータのうち、最新のデータ(s1より1つ前のデータ)
    where exists (
        select max(s3.year)
        from Sales2 as s3
        where s1.year > s3.year and
        s2.year = s3.year
    ) and
    s1.year - 1 = s2.year
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


year,sale
1993,52
1994,55


In [82]:
%%sql
-- 相関サブクエリの別解
select year, sale
from Sales2 as s1
where s1.year-1 = (
    select year
    from sales2 as s2
    -- s1より前のデータのうち、最新のデータ(s1より1つ前のデータ)
    where s2.year = (
        select max(s3.year)
        from Sales2 as s3
        where s1.year > s3.year
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


year,sale
1993,52
1994,55


In [79]:
%%sql
-- window関数を用いて、違う列のレコードと比較する
select year, sale
from (
    select year,sale,
        max(year) over (
            order by year
            rows between 1 preceding and 1 preceding
        ) as pre_year
    from Sales2
) as tmp
where year - 1 = pre_year

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


year,sale
1993,52
1994,55
